In [6]:
from config import get_snowpark_session
from snowflake.snowpark import functions as F

session = get_snowpark_session()

session_id: 14413629421
version: 1.37.0
database: "_DEV_ANALYTICS"
schema: "ASTAUS"
user: "astaus"


In [2]:
from faker import Faker

session.add_packages(["faker"])

In [3]:
fake = Faker()

@F.udf
def syn_first_name(col: str) -> str:
    fake.seed_locale("en_US", col)
    return fake.first_name()

@F.udf
def syn_last_name(col: str) -> str:
    fake.seed_locale("en_US", col)
    return fake.last_name()

In [4]:
df = session.table(["_dev_analytics", "accounts_db__astaus", "accounts"])
df.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ACCOUNT_ID"                            |"ACCOUNT_FIRST_NAME"              |"ACCOUNT_LAST_NAME"               |"ACCOUNT_EMAIL"                   |"INDVIDUAL_PARTY_KEY"  |"UPDATED_AT"                |"_LOADED_AT"                      |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|{028ff034-56ba-11f0-8960-325096b39f47}  |34b7da764b21d298ef307d04d8152dc5  |a66e44736e753d4533746ced572ca821  |f4135240818cba1d2ce7ea3c0430f3ba  |AGFSDGFSD              |2025-07-02 03:47:14.803742  |2025-08-27 18:25:13.952988+00:00  |
|{82278e27-39fb-46da-bd11-d4ea78c26334}  |3011cf8191156e

In [5]:
(df
    .with_column(
        "account_first_name",
        F.lower(syn_first_name(F.col("account_first_name")))
    )
    .with_column(
        "account_last_name",
        F.lower(syn_last_name(F.col("account_last_name")))
    )
    .select(
        "account_id",
        "account_first_name",
        "account_last_name",
        "indvidual_party_key",
    )
).show()

---------------------------------------------------------------------------------------------------------------
|"ACCOUNT_ID"                            |"ACCOUNT_FIRST_NAME"  |"ACCOUNT_LAST_NAME"  |"INDVIDUAL_PARTY_KEY"  |
---------------------------------------------------------------------------------------------------------------
|{028ff034-56ba-11f0-8960-325096b39f47}  |connor                |mccullough           |AGFSDGFSD              |
|{82278e27-39fb-46da-bd11-d4ea78c26334}  |rebecca               |herring              |BHGDFHTYR              |
|{14658546-d7dc-41ec-aa53-f340266d9d7b}  |david                 |burns                |CHGFDREWF              |
|{673c0869-b70a-495b-bbd3-53e15f537126}  |tracy                 |schneider            |DHHTRWFFD              |
|{c3eeac07-89bd-4d91-b561-21262f8e17aa}  |erika                 |burns                |EJUYTTRHG              |
--------------------------------------------------------------------------------------------------------